In [1]:
# Tetuan City Power Consumption: Train models and save assets for deployment
# This script is designed to be run once. Its purpose is to train the final
# machine learning models and save all the necessary components (assets)
# that a deployment application (like a Streamlit app or an API) will need
# to make live predictions without having to retrain.

# --- Core Library Imports ---
import pandas as pd  # For data manipulation and reading CSV files.
import numpy as np  # For numerical operations, especially with arrays.
import torch  # The main PyTorch library for building neural networks.
import torch.nn as nn  # Contains the building blocks for neural networks (layers, loss functions, etc.).
from torch.utils.data import TensorDataset, DataLoader  # For creating efficient data pipelines.
from sklearn.preprocessing import MinMaxScaler  # For scaling numerical features to a 0-1 range.
from sklearn import metrics  # For evaluating model performance (MAE, RMSE, R²).
import pickle  # Used for serializing and saving Python objects, like our scaler.

print("--- Starting Training and Asset Generation ---")

# --- 1. Data Loading and Preparation ---
# This section handles loading the raw data, cleaning it, and performing
# the necessary feature engineering steps.

try:
    # Load the dataset from the specified CSV file.
    df = pd.read_csv('Tetuan City power consumption.csv')
except FileNotFoundError:
    # Handle the case where the data file is not found.
    print("Error: 'Tetuan City power consumption.csv' not found.")
    exit()

# Clean column names by removing leading/trailing spaces and replacing single spaces with underscores.
df.columns = df.columns.str.strip().str.replace(' ', '_')
# Convert the 'DateTime' column from a string to a proper datetime object.
df['DateTime'] = pd.to_datetime(df['DateTime'])
# Set the 'DateTime' column as the index of the DataFrame, which is essential for time-series analysis.
df.set_index('DateTime', inplace=True)

# Define the list of raw features we will use for our model.
features_to_use = [
    'Temperature', 'Humidity', 'Wind_Speed', 'general_diffuse_flows',
    'Zone_1_Power_Consumption', 'Zone_2__Power_Consumption', 'Zone_3__Power_Consumption'
]
# Create a new DataFrame 'data' containing only the columns we need.
data = df[features_to_use].copy()

# --- Feature Engineering: Cyclical Time Features ---
# We convert cyclical time features (like hour of the day) into a format that
# machine learning models can understand. 23:00 is close to 00:00, and sine/cosine
# transformations represent this circular relationship.
data['hour_sin'] = np.sin(2 * np.pi * data.index.hour / 24.0)
data['hour_cos'] = np.cos(2 * np.pi * data.index.hour / 24.0)
data['dayofweek_sin'] = np.sin(2 * np.pi * data.index.dayofweek / 7.0)
data['dayofweek_cos'] = np.cos(2 * np.pi * data.index.dayofweek / 7.0)

# Store the final list of feature names for later use (e.g., in SHAP plots).
FEATURE_NAMES = data.columns.tolist()

# --- Data Splitting (Chronological) ---
# For time-series data, it's crucial to split the data chronologically to prevent
# data leakage (i.e., using future information to train the model).
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
# The remaining 15% will be the test set.
train_df = data.iloc[:train_size]
val_df = data.iloc[train_size:train_size + val_size]
test_df = data.iloc[train_size + val_size:]

# --- Scaling ---
# We scale the data to a range between 0 and 1. This helps the neural network
# train more effectively, as it prevents features with large values from dominating
# the learning process.
scaler = MinMaxScaler()
# IMPORTANT: We fit the scaler ONLY on the training data to avoid data leakage from the validation and test sets.
scaler.fit(train_df)
# Then, we transform all three sets using the fitted scaler.
train_scaled = scaler.transform(train_df)
val_scaled = scaler.transform(val_df)
test_scaled = scaler.transform(test_df)

# --- SAVE ASSET 1: THE SCALER ---
# The scaler is a critical asset. Any new data for prediction must be scaled
# in the exact same way as the training data. We save it using pickle.
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("\n--- Scaler has been fitted and saved to scaler.pkl ---")


# --- 2. Training Strategies: Early Stopping Class ---
# This helper class is used to monitor the model's performance on the validation set
# and stop the training process if it stops improving. This prevents overfitting
# and saves computational time.
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience  # How many epochs to wait for improvement before stopping.
        self.verbose = verbose  # If True, prints a message for each patience count.
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta  # A small value to consider an improvement.

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# ############################################################################
# This is the main part of the script. It loops through each Zone,
# trains a dedicated model, and saves its weights to a file.
# ############################################################################

# A dictionary to map user-friendly zone names to their column names in the DataFrame.
TARGET_ZONES = {
    'Zone 1': 'Zone_1_Power_Consumption',
    'Zone 2': 'Zone_2__Power_Consumption', # Note the double underscore from initial cleaning
    'Zone 3': 'Zone_3__Power_Consumption'
}

# Main loop to train a model for each zone.
for i, (zone_name, target_col_name) in enumerate(TARGET_ZONES.items()):
    print(f"\n{'='*80}")
    print(f"--- Processing {zone_name} (Target: {target_col_name}) ---")
    print(f"{'='*80}\n")

    # --- Sequence Creation Function ---
    # This function converts the flat time-series data into sequences (or windows)
    # that are suitable for an LSTM model.
    def create_sequences(input_data, lookback_window, target_column_index):
        X, y = [], []
        # Iterate through the data to create overlapping windows.
        for i in range(len(input_data) - lookback_window):
            # The input sequence (X) is a window of 'lookback_window' time steps.
            sequence = input_data[i:(i + lookback_window)]
            # The target (y) is the value of the target column immediately after the sequence.
            target = input_data[i + lookback_window, target_column_index]
            X.append(sequence)
            y.append(target)
        return np.array(X), np.array(y)

    # Define the lookback window: how many previous time steps the model should see to make a prediction.
    # 144 steps * 10 minutes/step = 1440 minutes = 24 hours.
    LOOKBACK_WINDOW = 144
    # Get the numerical index of our target column.
    TARGET_COL_IDX = train_df.columns.get_loc(target_col_name)

    # Create the sequences for training, validation, and testing.
    X_train, y_train = create_sequences(train_scaled, LOOKBACK_WINDOW, TARGET_COL_IDX)
    X_val, y_val = create_sequences(val_scaled, LOOKBACK_WINDOW, TARGET_COL_IDX)
    X_test, y_test = create_sequences(test_scaled, LOOKBACK_WINDOW, TARGET_COL_IDX)

    # --- Create PyTorch DataLoaders ---
    # DataLoaders handle batching, shuffling, and efficiently loading data for the model.
    # First, convert numpy arrays to PyTorch tensors.
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Create TensorDatasets, which pair the input tensors with their corresponding target tensors.
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # Define the batch size: how many sequences are processed at once.
    BATCH_SIZE = 64
    # Create the final DataLoader objects.
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # --- Model Architecture Definition ---
    # This class defines the structure of our neural network. It MUST be identical
    # in the deployment script to load the saved weights correctly.
    class BiLSTMForecaster(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
            super(BiLSTMForecaster, self).__init__()
            # Bidirectional LSTM layer processes the sequence in both forward and backward directions.
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                batch_first=True,  # This means the input tensor shape is (batch, sequence, feature).
                                dropout=dropout_rate, # Adds dropout for regularization.
                                bidirectional=True)
            # A final linear layer to map the LSTM output to our single prediction value.
            # The input size is hidden_size * 2 because the bidirectional LSTM concatenates
            # the outputs from the forward and backward passes.
            self.linear = nn.Linear(hidden_size * 2, output_size)

        def forward(self, x):
            # The forward pass defines how data flows through the network.
            lstm_out, _ = self.lstm(x)
            # We only care about the output of the very last time step in the sequence.
            last_time_step_out = lstm_out[:, -1, :]
            # Pass this last output through the linear layer to get the final prediction.
            out = self.linear(last_time_step_out)
            return out

    # --- Model Hyperparameters ---
    INPUT_SIZE = X_train.shape[2]  # The number of features in our input data.
    HIDDEN_SIZE = 64  # The number of neurons in each LSTM layer.
    NUM_LAYERS = 2  # The number of stacked LSTM layers.
    OUTPUT_SIZE = 1  # We are predicting a single value.
    DROPOUT_RATE = 0.2  # The dropout rate for regularization.

    # Instantiate the model.
    model = BiLSTMForecaster(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT_RATE)

    # --- Training Loop ---
    loss_function = nn.MSELoss()  # Mean Squared Error is a common loss function for regression.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam is an efficient and popular optimizer.
    early_stopper = EarlyStopping(patience=5, verbose=True) # Initialize our early stopper.
    # This scheduler will reduce the learning rate if the validation loss plateaus.
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    EPOCHS = 50  # The maximum number of epochs to train for.

    # Loop through each epoch.
    for epoch in range(EPOCHS):
        model.train()  # Set the model to training mode.
        running_loss = 0.0
        # Loop through batches of data from the train_loader.
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()  # Reset gradients.
            output = model(batch_X)  # Forward pass.
            loss = loss_function(output, batch_y)  # Calculate the loss.
            loss.backward()  # Backward pass (calculate gradients).
            optimizer.step()  # Update the model's weights.
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)

        model.eval()  # Set the model to evaluation mode.
        running_val_loss = 0.0
        with torch.no_grad():  # Disable gradient calculation for efficiency.
            for batch_X, batch_y in val_loader:
                output = model(batch_X)
                loss = loss_function(output, batch_y)
                running_val_loss += loss.item()
        avg_val_loss = running_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")
        
        # Step the scheduler and check for early stopping.
        scheduler.step(avg_val_loss)
        early_stopper(avg_val_loss, model)
        if early_stopper.early_stop:
            print("Early stopping triggered!")
            break
            
    print(f"\n--- Training Complete for {zone_name} ---")

    # --- Final Evaluation on the Unseen Test Set ---
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            output = model(batch_X)
            predictions.append(output.numpy())
            actuals.append(batch_y.numpy())
    
    # Concatenate results from all batches.
    predictions = np.concatenate(predictions).flatten()
    actuals = np.concatenate(actuals).flatten()

    # --- Inverse Transform Predictions to Original Scale ---
    # The model predicts scaled values. We need to inverse-transform them to get
    # the actual power consumption values in kW.
    # We create a dummy array of the correct shape, fill our target column with
    # the predictions, and then apply the inverse transform.
    dummy_array = np.zeros((len(predictions), len(data.columns)))
    dummy_array[:, TARGET_COL_IDX] = predictions
    predictions_inversed = scaler.inverse_transform(dummy_array)[:, TARGET_COL_IDX]

    # Do the same for the actual values to compare.
    dummy_array_actuals = np.zeros((len(actuals), len(data.columns)))
    dummy_array_actuals[:, TARGET_COL_IDX] = actuals
    actuals_inversed = scaler.inverse_transform(dummy_array_actuals)[:, TARGET_COL_IDX]

    # Calculate final performance metrics.
    mae = metrics.mean_absolute_error(actuals_inversed, predictions_inversed)
    rmse = np.sqrt(metrics.mean_squared_error(actuals_inversed, predictions_inversed))
    r2 = metrics.r2_score(actuals_inversed, predictions_inversed)
    print(f"\n--- Final Test Set Performance for {zone_name} ---")
    print(f"Mean Absolute Error (MAE): {mae:.2f} kW")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f} kW")
    print(f"R-squared (R²): {r2:.4f}")

    # --- SAVE ASSET 2: THE MODEL WEIGHTS ---
    # This is the final and most important asset. We save the model's learned
    # parameters (state_dict) to a .pth file.
    model_filename = f'model_{zone_name.replace(" ", "_")}.pth'
    torch.save(model.state_dict(), model_filename)
    print(f"--- Model for {zone_name} saved to {model_filename} ---")

print("\n--- All models trained and assets saved. Deployment preparation is complete. ---")



--- Starting Training and Asset Generation ---

--- Scaler has been fitted and saved to scaler.pkl ---

--- Processing Zone 1 (Target: Zone_1_Power_Consumption) ---

Epoch 1/50 | Train Loss: 0.005940 | Validation Loss: 0.000806
Epoch 2/50 | Train Loss: 0.000575 | Validation Loss: 0.001401
EarlyStopping counter: 1 out of 5
Epoch 3/50 | Train Loss: 0.000438 | Validation Loss: 0.000272
Epoch 4/50 | Train Loss: 0.000359 | Validation Loss: 0.000236
Epoch 5/50 | Train Loss: 0.000297 | Validation Loss: 0.000244
EarlyStopping counter: 1 out of 5
Epoch 6/50 | Train Loss: 0.000278 | Validation Loss: 0.000223
Epoch 7/50 | Train Loss: 0.000236 | Validation Loss: 0.000149
Epoch 8/50 | Train Loss: 0.000227 | Validation Loss: 0.000149
Epoch 9/50 | Train Loss: 0.000203 | Validation Loss: 0.000146
Epoch 10/50 | Train Loss: 0.000204 | Validation Loss: 0.000126
Epoch 11/50 | Train Loss: 0.000189 | Validation Loss: 0.000162
EarlyStopping counter: 1 out of 5
Epoch 12/50 | Train Loss: 0.000184 | Validation 